# 02_05: Solution: Evaluate Image Quality

### Install the necesarry libraries.

In [1]:
pip install tensorflow

In [2]:
pip install tensorflow_gan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 30.7 MB/s eta 0:00:00


In [3]:
pip install scikit-image

### Calculate the FID score

In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input
from skimage import io
from scipy.linalg import sqrtm

In [5]:
def load_and_preprocess_images(directory, target_size=(299, 299)):
    images = []
    for filename in os.listdir(directory):
      if filename.endswith(".png"):
        image=io.imread(os.path.join(directory,filename))
        if image.shape[-1] == 4:#some png images have 4 channels(RGBA)
          image = image[...,:3] #remove alpha channel if present
        image = tf.image.resize(image,target_size)
        image = preprocess_input(image.numpy())#converts the tensorflow image into numpy array
        images.append(image)
    return np.array(images)

def calculate_fid(model,images1,images2):
  #calculate activations
  act1 = model.predict(images1)
  act2 = model.predict(images2)

  #Calculate mean and covariance statistics
  mu1,sigma1 = act1.mean(axis=0),np.cov(act1,rowvar=False)
  mu2,sigma2 = act2.mean(axis=0),np.cov(act2,rowvar=False)

  #Calculate sum squared difference between means
  ssdiff = np.sum((mu1-mu2)**2.0)

  #Calculate sqrt of product between covariances
  covmean = sqrtm(sigma1.dot(sigma2))

  # Check and correct imaginary numbers from sqrt
  if np.iscomplexobj(covmean):
    covmean = covmean.real

  #Calculate FID score
  fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
  return fid

In [6]:
!git clone https://github.com/LinkedInLearning/evaluating-and-debugging-generative-ai-3808127.git

Cloning into 'evaluating-and-debugging-generative-ai-3808127'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 91 (delta 30), reused 71 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (91/91), 2.15 MiB | 49.96 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [7]:
%cd evaluating-and-debugging-generative-ai-3808127/02_05
!ls


/content/evaluating-and-debugging-generative-ai-3808127/02_05
02_05-after.ipynb   fake_images  sample_images.zip
02_05-before.ipynb  real_images


In [8]:
# Paths to your dataset folders
real_images_path = 'real_images/'
generated_images_path = 'fake_images/'

STEP 1: Load and Preprocess your images using the load_and_preprocess_images function

In [9]:
# Load and Preprocess images
real_images = load_and_preprocess_images(real_images_path)

In [10]:
generated_images = load_and_preprocess_images(generated_images_path)

Step 2: Load InceptionV3 model with the following parameters: include_top=False,pooling="avg",input_shape=(299,299,3)

In [11]:
#load the inception model
inception_model = InceptionV3(include_top=False,pooling="avg",input_shape=(299,299,3))

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Step3: Calculate and print the FID using calculate_fid function

In [12]:
# Calculate FID
fid_score = calculate_fid(inception_model,real_images,generated_images)
print(f'FID score: {fid_score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
FID score: 68.25395812893382


/tmp/ipython-input-2624114778.py:26: LinAlgWarning: Matrix is singular. The result might be inaccurate or the array might not have a square root.
  covmean = sqrtm(sigma1.dot(sigma2))
